# Enron Fraud Detection Using Machine Learning

## Motivation and Goal

After the Enron scandal, a unprecedented amount of information became available to the public about the internal workings of Enron. This information included email correspondences between employees and officers with in the company, and their financial compensation. During the investigation, many people were identified as persons of interest (POI).

This exercise applies machine learning techniques on some of the information that was made available and see if the POI could be identified based on the information that was available for that person. Machine learning techniques analyzes patterns that might be missed by human investigators. It would mean that these patterns might be missed by future perpetrators of frauds. If the technique used here can be generalized, it would mean that there are more tools that investigators can use to uncover fraud.

One requirement for completing the exercise is to achieve at least a score of 0.3 in recall and precision scores for the classifier. Using an naive approach, it's fairly challenging to achieve the required recall and precision scores. As the result, I took it as a meta-challenge to try to beat it using a simple, straightforward, and from first principles approach.

## Initial Data Exploration

There are initially 146 persons, each with 21 features, in our dataset. Out of the 146 candidates, there are 18 persons of interest (POIs). However, everyone has at least one field that had a "NaN" value. I had to be selective in which features I chose to include in my dataset so that I had both enough features and enough data points  for the algorithms to give me a good result.

## Feature Selection

Like any good detective, I used the model of means, motive and opportunity to trying to catch the POI in my approach.

Motive was fairly easy to reason. The perpetrators were artificially raising the stock price. They must have significant stockholding to make it worthwhile. Conversely, someone without significant stockholder might be an unlikely perpetrator, since it does not benefit them to participate in the fraud. As the result, I chose "total_stock_value" as one of my features.

One reason for wanting to employ fraud to make more money might be that the perpetrators has a unusually large needs for the money. They might be living beyond their means. It might show up as a part of their company expenses too. So, I also included "expenses" as one of my features.

As part of the means, the perpetrators must communicate with each other somehow, and must coordinate their action more tightly than people would regularly do in a large company. At the time of the Enron scandal, it was still in the relatively "early days" of the internet. People are probably less conscious of their email security. People might be emailing each other in a much more carefree manner. However, this would mean that everyone is probably emailing much more. I would expect that there would be a unusually large amount of email going both to and from the various POI. To capture this information succinctly, I will create two features: one would calculate emails sent to POIs as a percentage of total emails sent, and one would calculate emails received from POIs as a percentage of total emails received. 

In addition to email, the perpetrators might also meet face to face. I decided it would be benefitial to include ""shared_receipt_with_poi" as a feature.

Opportunity is harder establish. I would expect most people involved in the scandal to be in position of power, or key positions within the company. If not, it's unlikely that the scam would have had the success it did. However, the position or job titles for the various people involved are not given as part of the data set. I would expect people in power or key positions, who had the power to pull off the scam, would have a lot of responsibilities with in the company. They would be highly valued by the company, and would be well compensated in one form or another. In order to simply the model, I chose to simply use "total_payments" as a feature.

## Outlier Removal

### Removing Data Points with Missing Value

I removed the data points with "NaN" values in the features that I am interested in. The result was that my dataset was reduced to 57 members. I had 14 POIs among those 57.

### Removing Outlier Values

The identified POI is a small portion of the total population. I had to be careful about removing the outlying values. In addition, I expected that many POI would be considered outliers in many models. As the result, I took an different approach.

I reasoned that if the POI are most likely outliers from the view of the total population, the non-POI members would be much more "normal" or uniform. After performing an naive linear regression on my data as is, I chose to remove hardest to classify 50% of the population who were non-POI. As the result, I had a 50% POI Ratio.

## Algorithm Choice

In the initial naive attempt, I used the default Gaussian Naive Bayes algorithm without modification. It performed poorly, even after feature selection and outlier removal.

I believe that I had a clear picture of the shape of my data based on feature selection and outlier removal. I had removed the hard to classify non-POI. As the result, I expected the non-POI population would be relatively uniform and closely clumped. The POI on the other hands, I expected would be a loose cloud outside of the non-POI region. I just needed to pick an algorithm that could simply separate the two regions, and I would have achieved my goal of having a simple approach to obtaining high precision and recall scores.

I believed my the best choices were SVM and Decision Tree. I believe a k-mean algorithm would misidentify more points as the POI points would be too loosely spread out. My final choice was to use a Decision Tree.

Based on earlier trials, a min-max scaler seemed to improve the result of my classier. As the result, I decided to use a min-max scaler for all of my features.

## Algorithm Tuning

Machine learning approach and algorithms can be considered general approaches to solve a particular class of problems. Their default parameters are set to be generally applicable for many specific problems within their class of problems. As the result, it's usually necessary to make adjustment to an algorithm's parameter to get the best performance out of an algorithm. It's like putting a car into the proper gear.

I choose to use "entropy" for my splitting criterion, since I expect that I only need a simple algorithm based on my outlier removal strategy. Based on that idea, I experimented lowering the "max_depth" of my decision tree. As expected, I got my best validation score when I lowered it to 2. Naturally, the performance became much worse when I lowered the "max_depth" to 1.

```
# max_depth at default
Accuracy: 0.79450
Precision: 0.40284
Recall: 0.48300
F1: 0.43929
F2: 0.46451

# max_depth = 4
Accuracy: 0.79533
Precision: 0.42050
Recall: 0.60300
F1: 0.49548
F2: 0.55484

# max_depth = 2
Accuracy: 0.80783
Precision: 0.45651
Recall: 0.80300
F1: 0.58209
F2: 0.69717

# max_depth = 1
Accuracy: 0.80583
Precision: 0.18631
Recall: 0.04900
F1: 0.07759
F2: 0.05747
```

I initially experimented with "min_samples_split" value. However, since my final data set is relatively small. My algorithm seems to perform worse when it's set too high. With the lower "max_depth", I had to set it very high to make a difference, and it's usually detrimental. As the result, I am leaving it at the default value.

## Evaluation

I used precision and recall as my evaluation metrics. Recall and precision can be considered a way to quantify type 1 and type 2 errors. Recall measures the rate at which a correct label was applied to all the members that should have had this label. Precision measures the rate at which a correct label was applied correctly. In this context, recall for POI label would be the percentage that all POIs was correctly labeled as such. Precision would be the percentage of data points labeled as POI was correctly labeled.

A related concept for precision and recall is the idea of a F1 score. Since both precision and recall are important, we do not want to train algorithm to value one score at the expense of the other. In this case, F1 score is used to bring balance between precision and recall.

## Validation

The reason for training our model is to generalize to cases that we have no seen before. With enough data and training, we can bring precision and recall scores to reach a very high level. However, we will be risking overfitting our model. In that case, althought we have a good precision and recall score for our training data, we cannot generalize it to a different data set. As the result, we must also balance between fitting our algorithm to the data we have, and the data we expect to use the algorithm on.

As the result, even with the data that we have, we want to split it into training sets and testing sets to avoid overfitting our algorithm.

## Result

I was able to pass the required 0.3 mark in precision and recall without requiring to fine-tuning the Decision Tree algorithm too much and I was able used a reasoned approach to the project, which I took as having achieved my meta-challenge. Based on "tester.py" result, my scores was:

```
Accuracy: 0.80783
Precision: 0.45656
Recall: 0.80400
F1: 0.58240
F2: 0.69780
```

## Reflection

I believe my result is largely attributable to my outlier removal strategy. Thinking through the process enabled me to get a better feel for the data, and subsequent tests bore our my assumptions about the shape of my data. It was quite satisfying to beat the required precision and recall using such a straightforward approach.

In a real world scenario, the required F1 scores would probably be much higher. I think I would have chosen to use a SVM in such a case. However, if I were to continue using a Decision Tree classifer, I think there are still a few low hanging fruits that I can use to improve my classifier. By having removed many ambiguous non-POI data points before training, I can afford to increase the variance of the classifier. I can do so my increasing the "min_sample_split" or by decreasing the "max_depth" of the classifier.